In [3]:
import h2o
import zipfile
import os
import sys
from pyspark.sql import SparkSession
from IPython.display import display
from pyspark.sql.functions import regexp_extract, col, split, udf, \
                                 trim, when, from_unixtime, unix_timestamp, minute, hour, datediff, lit, array,\
                                 to_date
from pyspark.sql import functions as F
from pyspark.sql.types import IntegerType, StringType, BooleanType, ArrayType, StructType, StructField, LongType, TimestampType
import datetime
import argparse
import json
import glob, os, shutil
import pandas as pd
from pandas.io.json import json_normalize
from pyspark import SparkContext
from time import sleep
from math import floor


pd.options.display.max_columns = 99

sc = SparkContext()

spark = SparkSession \
        .builder \
        .appName("Data ETL") \
        .config("spark.some.config.option", "some-value") \
        .getOrCreate()
        
display(spark.version)

'2.1.1'

In [2]:
! aws s3 ls

2017-05-28 11:55:59 flight.machine.bill
2017-06-25 09:39:16 flight.pq.10
2017-06-25 05:44:15 flight.pq.working
2017-05-02 03:08:49 flight.price
2017-05-12 13:28:06 flight.price.11


In [3]:
! aws s3 ls flight.pq.10

                           PRE flight_15_10/
                           PRE flight_15_11/
                           PRE flight_15_12/
                           PRE flight_15_14/
                           PRE flight_15_5/
                           PRE flight_15_7/
                           PRE flight_15_9/
                           PRE flight_16_10/
                           PRE flight_16_11/
                           PRE flight_16_13/
                           PRE flight_16_14/
                           PRE flight_16_5/
                           PRE flight_16_6/
                           PRE flight_16_8/
                           PRE flight_1_10/
                           PRE flight_1_13/
                           PRE flight_1_5/
                           PRE flight_1_6/
                           PRE flight_1_7/
                           PRE flight_1_9/
                           PRE flight_2_11/
                           PRE flight_2_5/
                           PR

In [11]:
! aws s3 ls flight.pq.working

                           PRE flight_10_14__flight_10_14_price_2017-05-16.pq/
                           PRE flight_10_14__flight_10_14_price_2017-05-17.pq/
                           PRE flight_10_14__flight_10_14_price_2017-05-18.pq/
                           PRE flight_10_14__flight_10_14_price_2017-05-19.pq/
                           PRE flight_10_14__flight_10_14_price_2017-05-20.pq/
                           PRE flight_10_14__flight_10_14_price_2017-05-21.pq/
                           PRE flight_10_14__flight_10_14_price_2017-05-22.pq/
                           PRE flight_10_14__flight_10_14_price_2017-05-23.pq/
                           PRE flight_10_14__flight_10_14_price_2017-05-24.pq/
                           PRE flight_10_14__flight_10_14_price_2017-05-25.pq/
                           PRE flight_10_14__flight_10_14_price_2017-05-26.pq/
                           PRE flight_10_14__flight_10_14_price_2017-05-27.pq/
                           PRE flight_10_14__flight_

In [12]:
sc._jsc.hadoopConfiguration().set("fs.s3n.awsAccessKeyId", "")
sc._jsc.hadoopConfiguration().set("fs.s3n.awsSecretAccessKey", "")

In [15]:
! pwd

/home/ubuntu/work/flight


In [19]:
! aws s3 cp s3://flight.pq.working /home/ubuntu/s3/flight.pq.working --recursive

download: s3://flight.pq.working/flight_10_14__flight_10_14_price_2017-05-16.pq/._SUCCESS.crc to ../../s3/flight.pq.working/flight_10_14__flight_10_14_price_2017-05-16.pq/._SUCCESS.crc
download: s3://flight.pq.working/flight_10_14__flight_10_14_price_2017-05-16.pq/_SUCCESS to ../../s3/flight.pq.working/flight_10_14__flight_10_14_price_2017-05-16.pq/_SUCCESS
download: s3://flight.pq.working/flight_10_14__flight_10_14_price_2017-05-18.pq/_SUCCESS to ../../s3/flight.pq.working/flight_10_14__flight_10_14_price_2017-05-18.pq/_SUCCESS
download: s3://flight.pq.working/flight_10_14__flight_10_14_price_2017-05-16.pq/.part-00000-cfdb655d-4326-431d-86ea-497591ae0326.snappy.parquet.crc to ../../s3/flight.pq.working/flight_10_14__flight_10_14_price_2017-05-16.pq/.part-00000-cfdb655d-4326-431d-86ea-497591ae0326.snappy.parquet.crc
download: s3://flight.pq.working/flight_10_14__flight_10_14_price_2017-05-17.pq/_SUCCESS to ../../s3/flight.pq.working/flight_10_14__flight_10_14_price_2017-05-17.pq/_SUCCE

In [21]:
# ! rm -rf /home/ubuntu/s3/flight.pq.10

In [20]:
! aws s3 cp s3://flight.pq.10 /home/ubuntu/s3/flight.pq.10 --recursive

download: s3://flight.pq.10/flight_15_10/_SUCCESS to ../../s3/flight.pq.10/flight_15_10/_SUCCESS
download: s3://flight.pq.10/flight_15_12/_SUCCESS to ../../s3/flight.pq.10/flight_15_12/_SUCCESS
download: s3://flight.pq.10/flight_15_10/.part-00000-7064615b-2139-4166-b801-3444d2d907b0.snappy.parquet.crc to ../../s3/flight.pq.10/flight_15_10/.part-00000-7064615b-2139-4166-b801-3444d2d907b0.snappy.parquet.crc
download: s3://flight.pq.10/flight_15_14/_SUCCESS to ../../s3/flight.pq.10/flight_15_14/_SUCCESS
download: s3://flight.pq.10/flight_15_12/.part-00000-517c6593-bdf5-476a-bd9d-1edf18864b4c.snappy.parquet.crc to ../../s3/flight.pq.10/flight_15_12/.part-00000-517c6593-bdf5-476a-bd9d-1edf18864b4c.snappy.parquet.crc
download: s3://flight.pq.10/flight_15_14/._SUCCESS.crc to ../../s3/flight.pq.10/flight_15_14/._SUCCESS.crc
download: s3://flight.pq.10/flight_15_12/._SUCCESS.crc to ../../s3/flight.pq.10/flight_15_12/._SUCCESS.crc
download: s3://flight.pq.10/flight_15_10/._SUCCESS.crc to ../../s

In [4]:
flight = spark.read.parquet("/home/ubuntu/s3/flight.pq.10/*")

In [5]:
flight.printSchema()

root
 |-- basic: struct (nullable = true)
 |    |-- from_city_name: string (nullable = true)
 |    |-- search_date: string (nullable = true)
 |    |-- start_date: string (nullable = true)
 |    |-- stay_days: string (nullable = true)
 |    |-- table_name: string (nullable = true)
 |    |-- task_id: string (nullable = true)
 |    |-- to_city_name: string (nullable = true)
 |    |-- trip: string (nullable = true)
 |    |-- version: string (nullable = true)
 |-- flight_list: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- airline_code: string (nullable = true)
 |    |    |-- airline_codes: array (nullable = true)
 |    |    |    |-- element: string (containsNull = true)
 |    |    |-- arr_time: string (nullable = true)
 |    |    |-- check_bag_inc: boolean (nullable = true)
 |    |    |-- company: string (nullable = true)
 |    |    |-- dep_time: string (nullable = true)
 |    |    |-- duration: string (nullable = true)
 |    |    |-- flight_code: st

In [29]:
flight.select("flight_list")

DataFrame[flight_list: array<struct<airline_code:string,airline_codes:array<string>,arr_time:string,check_bag_inc:boolean,company:string,dep_time:string,duration:string,flight_code:string,flight_number:string,id:string,index:bigint,plane:string,power:boolean,price:double,price_code:string,search_date:string,span_days:bigint,stop:bigint,stop_info:string,ticket_left:bigint,video:boolean,wifi:boolean>>]

In [6]:
flight.count()

1246935

In [7]:
flight_v1_1 = spark.read.parquet("/home/ubuntu/s3/flight.pq.working/*")

In [8]:
flight_v1_1.count()

4884930

In [9]:
flight_v1_1.printSchema()

root
 |-- price: double (nullable = true)
 |-- version: string (nullable = true)
 |-- searchDate: date (nullable = true)
 |-- tableName: string (nullable = true)
 |-- task_id: long (nullable = true)
 |-- currencyCode: string (nullable = true)
 |-- fromCity: string (nullable = true)
 |-- toCity: string (nullable = true)
 |-- trip: string (nullable = true)
 |-- depDate: date (nullable = true)
 |-- retDate: date (nullable = true)
 |-- stayDays: integer (nullable = true)
 |-- departureTime: string (nullable = true)
 |-- arrivalTime: string (nullable = true)
 |-- departureTime_leg2: string (nullable = true)
 |-- arrivalTime_leg2: string (nullable = true)
 |-- airlineName: string (nullable = true)
 |-- airlineName_leg2: string (nullable = true)
 |-- duration_m: double (nullable = true)
 |-- duration_m_leg2: double (nullable = true)
 |-- flight_code: string (nullable = true)
 |-- plane: string (nullable = true)
 |-- stops: byte (nullable = true)
 |-- stops_leg2: byte (nullable = true)
 |-- st

In [10]:
display(flight_v1_1
        .groupBy("fromCity", "toCity", "trip", "version")
        .count()
        .orderBy("fromCity", "toCity", "trip", "version").show())

+--------+------+----+-------+-------+
|fromCity|toCity|trip|version|  count|
+--------+------+----+-------+-------+
| Chengdu|Taipei|   1|    1.1| 458329|
| Chengdu|Taipei|   2|    1.1|1813729|
| Chengdu|sydney|   1|    1.1| 472857|
| Chengdu|sydney|   2|    1.1|2140015|
+--------+------+----+-------+-------+



None

In [11]:
display(flight
        .groupBy("basic")
        .count()
        .orderBy("basic").show(1000, truncate=False))
# 8829 records with basic = null

+---------------------------------------------------------------------------+-----+
|basic                                                                      |count|
+---------------------------------------------------------------------------+-----+
|null                                                                       |8829 |
|[Bangkok,2017-04-25,2017-04-26,0,flight_15_10_price,13501,Chengdu,1,1.0]   |1    |
|[Bangkok,2017-04-25,2017-04-26,0,flight_15_11_price,14401,Chongqing,1,1.0] |1    |
|[Bangkok,2017-04-25,2017-04-26,0,flight_15_12_price,15301,Wuhan,1,1.0]     |1    |
|[Bangkok,2017-04-25,2017-04-26,0,flight_15_14_price,17101,Taipei,1,1.0]    |1    |
|[Bangkok,2017-04-25,2017-04-26,0,flight_15_5_price,9001,beijing,1,1.0]     |1    |
|[Bangkok,2017-04-25,2017-04-26,0,flight_15_7_price,10801,guangzhou,1,1.0]  |1    |
|[Bangkok,2017-04-25,2017-04-26,0,flight_15_9_price,12601,hongkong,1,1.0]   |1    |
|[Bangkok,2017-04-25,2017-04-26,14,flight_15_10_price,13503,Chengdu,2,1.0]  

None

In [13]:
flight2 = (flight.withColumn("aFlight", F.explode("flight_list"))
                 .drop("flight_list"))

In [14]:
flight2.printSchema()

root
 |-- basic: struct (nullable = true)
 |    |-- from_city_name: string (nullable = true)
 |    |-- search_date: string (nullable = true)
 |    |-- start_date: string (nullable = true)
 |    |-- stay_days: string (nullable = true)
 |    |-- table_name: string (nullable = true)
 |    |-- task_id: string (nullable = true)
 |    |-- to_city_name: string (nullable = true)
 |    |-- trip: string (nullable = true)
 |    |-- version: string (nullable = true)
 |-- flight_list: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- airline_code: string (nullable = true)
 |    |    |-- airline_codes: array (nullable = true)
 |    |    |    |-- element: string (containsNull = true)
 |    |    |-- arr_time: string (nullable = true)
 |    |    |-- check_bag_inc: boolean (nullable = true)
 |    |    |-- company: string (nullable = true)
 |    |    |-- dep_time: string (nullable = true)
 |    |    |-- duration: string (nullable = true)
 |    |    |-- flight_code: st

In [15]:
flight2.count()

116609561

In [20]:
flight = flight2.drop("flight_list")

In [29]:
flight.show(2)

Py4JNetworkError: An error occurred while trying to connect to the Java server (127.0.0.1:46754)

In [22]:
flight.printSchema()

root
 |-- basic: struct (nullable = true)
 |    |-- from_city_name: string (nullable = true)
 |    |-- search_date: string (nullable = true)
 |    |-- start_date: string (nullable = true)
 |    |-- stay_days: string (nullable = true)
 |    |-- table_name: string (nullable = true)
 |    |-- task_id: string (nullable = true)
 |    |-- to_city_name: string (nullable = true)
 |    |-- trip: string (nullable = true)
 |    |-- version: string (nullable = true)
 |-- aFlight: struct (nullable = true)
 |    |-- airline_code: string (nullable = true)
 |    |-- airline_codes: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- arr_time: string (nullable = true)
 |    |-- check_bag_inc: boolean (nullable = true)
 |    |-- company: string (nullable = true)
 |    |-- dep_time: string (nullable = true)
 |    |-- duration: string (nullable = true)
 |    |-- flight_code: string (nullable = true)
 |    |-- flight_number: string (nullable = true)
 |    |-- id: string (n

In [23]:
# for one way trips, display None in stay_days
def correct_stay_days(trip, stay_days):
    if trip == '1':
        return None
    else:
        return int(stay_days)

correct_stay_days_UDF = udf(correct_stay_days, IntegerType())

def correct_tickets_left(noOfTicketsLeft):
    if noOfTicketsLeft == 0:
        return 99
    else:
        return noOfTicketsLeft
    
correct_tickets_left_UDF = udf(correct_tickets_left, IntegerType())

timeFmt = "yyyy-MM-dd'T'HH:mm:ss.SSS"

take_all_level2_str = udf(lambda rows, a, b: None if rows is None else [None if row is None else row[a][b] for row in rows], ArrayType(StringType()))
take_all_level1_str = udf(lambda rows, a: None if rows is None else [None if row is None else row[a] for row in rows], ArrayType(StringType()))

# Check list of variables to be processed
## basic
from_city_name: string (nullable = true)  
search_date: string (nullable = true)  
~~start_date: string (nullable = true)~~  
~~stay_days: string (nullable = true)~~  
table_name: string (nullable = true)  
task_id: string (nullable = true)  
to_city_name: string (nullable = true)  
trip: string (nullable = true)  
version: string (nullable = true)  
 
## flight_list
airline_code: string (nullable = true)  
airline_codes: array (nullable = true)  
 |-- element: string (containsNull = true)  
arr_time: string (nullable = true)  
check_bag_inc: boolean (nullable = true)  
company: string (nullable = true)  
dep_time: string (nullable = true)  
duration: string (nullable = true)  
flight_code: string (nullable = true)  
flight_number: string (nullable = true)  
id: string (nullable = true)  
index: long (nullable = true)  
plane: string (nullable = true)  
power: boolean (nullable = true)  
price: double (nullable = true)  
price_code: string (nullable = true)  
search_date: string (nullable = true)  
span_days: long (nullable = true)  
stop: long (nullable = true)  
stop_info: string (nullable = true)  
ticket_left: long (nullable = true)  
video: boolean (nullable = true)  
wifi: boolean (nullable = true)  

In [26]:
flight2 = (flight.withColumn('stayDays', correct_stay_days_UDF(flight.basic.trip, flight.basic.stay_days))
#                  .drop('stay_days')           
#                  .withColumnRenamed('start_date', 'depDate')                 
                 .withColumn('depDate', flight.basic.start_date)                 
                 .withColumn('depDate', to_date('depDate'))
                 .selectExpr('*', 'date_add(depDate, stayDays) as retDate')# this is when the return trip starts, might arrive a day later
                 .withColumn('fromCity', flight.basic.from_city_name)
                 .withColumn('toCity', flight.basic.to_city_name)                 
                 .withColumn('search_date', flight.basic.search_date)                 
                 .withColumn('searchDate', to_date('search_date'))
                 .withColumn('airlineName', flight.aFlight.company)                 
                 .withColumn('departureTime', flight.aFlight.dep_time)                                  
                 .withColumn('arrivalTime', flight.aFlight.arr_time)                                                   
#                  .withColumn('duration_h', split(flight.duration,'h').getItem(0))
#                  .withColumn('duration_m', F.substring_index(split(flight.duration,'h').getItem(1), 'm', 1))
# #                  .withColumn('duration', F.struct(col('duration_h'), col('duration_m')))
#                  .withColumn('duration_m', (col('duration_h')*60 + col('duration_m')))
#                  .drop('duration', 'duration_h', 'flight_number')
#                  .withColumnRenamed('price_code', 'currencyCode')                                  
#                  .withColumnRenamed('stop', 'stops')
#                  .withColumn('stops', col('stops').cast('byte')) 
#                  .withColumn('stop_info', split(col('stop_info'), ';'))
# #                  .withColumn('stop_duration', take_all_duration_UDF(col('stop_info')))
#                  .withColumn('noOfTicketsLeft', correct_tickets_left_UDF('ticket_left'))
#                  .withColumn('noOfTicketsLeft', col('noOfTicketsLeft').cast('byte')) 
#                 .drop('ticket_left')
#                .withColumnRenamed('table_name', 'tableName')
#                .withColumn('task_id', col('task_id').cast('long')) 
#                .withColumn('span_days', col('span_days').cast('integer')) 
#                 .select('price', 'version', 'searchDate', 'tableName', 'task_id', 'currencyCode', 
#                         'fromCity', 'toCity', 'trip', 'depDate', 'retDate',
#                         'stayDays', 
#                        'departureTime', 'arrivalTime', 
#                         'airlineName',  'duration_m', 
#                         'flight_code', 'plane', 'stops', 'noOfTicketsLeft',
#                        'airline_code', 'airline_codes',
#                        'stop_info', 'span_days', 'power', 'video', 'wifi')                #'stop_duration', 
          )


In [28]:
flight2.show(2)

Py4JNetworkError: An error occurred while trying to connect to the Java server (127.0.0.1:46754)

In [27]:
flight2.limit(10).toPandas().head()

----------------------------------------
Exception happened during processing of request from ('127.0.0.1', 41590)


Traceback (most recent call last):
  File "/home/ubuntu/anaconda3/lib/python3.5/socketserver.py", line 313, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/home/ubuntu/anaconda3/lib/python3.5/socketserver.py", line 341, in process_request
    self.finish_request(request, client_address)
  File "/home/ubuntu/anaconda3/lib/python3.5/socketserver.py", line 354, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/home/ubuntu/anaconda3/lib/python3.5/socketserver.py", line 681, in __init__
    self.handle()
  File "/home/ubuntu/spark-2.1.1-bin-hadoop2.7/python/pyspark/accumulators.py", line 235, in handle
    num_updates = read_int(self.rfile)
  File "/home/ubuntu/spark-2.1.1-bin-hadoop2.7/python/pyspark/serializers.py", line 577, in read_int
    raise EOFError
EOFError


----------------------------------------


Py4JError: An error occurred while calling o9.setCallSite